In [13]:
import numpy as np
import pandas as pd

In [14]:
data = pd.read_csv("/home/buhturi/Desktop/datasets/Offensive_dataset.csv")
data.head(10)

,tweet,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",1
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",1
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,1
8,""" &amp; you might not get ya bitch back &amp; ...",1
9,""" @rhythmixx_ :hobbies include: fighting Maria...",1


In [15]:
tweets = list(data['tweet'])
labels = list(data['label'])

In [16]:
len(tweets)

24783

In [17]:
test_n = int(len(tweets) * 0.20)
train_n = len(tweets) - test_n
test_n


4956

In [18]:
train_x = tweets[:train_n] 
test_x = tweets[train_n:]
train_y = labels[:train_n]
test_y = labels[train_n:]

In [19]:
test_y.shape

AttributeError: 'list' object has no attribute 'shape'

In [20]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer 

In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/buhturi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
def process_tweet(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    stopwords_english = stopwords.words('english')
    tweet_clean = []
    for word in tweet_tokens:
        if(word not in stopwords_english and  word not in string.punctuation):
            tweet_clean.append(word)

    stemmer = PorterStemmer() 
    tweet_stem = []
    for word in tweet_clean:
        word_stem = stemmer.stem(word)
        tweet_stem.append(word_stem) 

    return tweet_stem

In [23]:
def count_tweets(result, tweets, ys):
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in result:
                result[pair] += 1
            else:
                result[pair] = 1
    return result

In [24]:
freqs = count_tweets({}, train_x, train_y)

In [25]:
def lookup(freqs, word, label):
    result = 0
    for pair in freqs.keys():
        if(pair == (word, label)):
            result +=freqs[pair]
            
    return result

In [26]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    N_offensive = 0
    N_not_offensive = 0
    for pair in freqs.keys():
        if(pair[1] > 0):
            N_offensive += freqs[pair]
        else:
            N_not_offensive += freqs[pair]
    
    
    D = len(train_y)
    D_offensive = np.sum(train_y)
    D_not_offensive = D - D_offensive
    
    logprior = np.log(D_offensive) - np.log(D_not_offensive)
    
    for word in vocab :
        freq_offensive = lookup(freqs,word,1)
        freq_not_offensive = lookup(freqs ,word ,0)
        
        p_w_offensive = (freq_offensive+1) / (N_offensive + V)
        p_w_not_offensive = (freq_not_offensive+1) / (N_not_offensive + V)

        loglikelihood[word] = np.log(p_w_offensive / p_w_not_offensive)
    
    return logprior, loglikelihood
    

In [27]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    
    word_l = list(process_tweet(tweet))
    p = logprior
    
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
            
    return p

In [28]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

1.5861957167591054
15017


In [29]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0
    y_hats = []
    for tweet in test_x :
        if(naive_bayes_predict(tweet, logprior, loglikelihood) > 0):
            y_hats.append(1)
        else:
            y_hats.append(0)
    
    error = np.mean(y_hats) - np.mean(test_y) 
    accuracy = 1 - error
    
    return accuracy

In [30]:
print("accuracy = %0.4f" % (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

accuracy = 0.9560


In [31]:
#test any qoute 
my_tweet = 'In the Yankees draft room, they had a huge picture of Greg Campbell with the message, "this is the makeup we\'re looking for."'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)
if(p > 0) : print("offensive language")
else : print("not offensive language")

-5.317353066505897
not offensive language


In [32]:
#test any qoute 
my_tweet = 'hey bitch!'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)
if(p > 0) : print("offensive language")
else : print("not offensive language")

7.786202612612794
offensive language
